In [1]:
import tensorflow as tf
import numpy as np


In [2]:
char_rdic = ['h', 'e', 'l', 'o']
char_dic = {w: i for i, w in enumerate(char_rdic)}
x_data = np.array([ [1, 0, 0, 0],
                  [0, 1, 0, 0],
                  [0, 0, 1, 0], 
                  [0, 0, 1, 0]], dtype='f')
sample = [char_dic[c] for c in 'hello']

In [3]:
char_vocab_size = len(char_dic) + 1
rnn_size = char_vocab_size # 1 hot coding. one of 4
time_step_size = 4 # 'hell' -> predict 'ello'
batch_size = 1 # one sample

In [4]:
rnn_cell = tf.nn.rnn_cell.BasicRNNCell(num_units = rnn_size)
#rnn_cell = tf.nn.rnn_cell.BasicLSTMCell(rnn_size)
#rnn_cell = tf.nn.rnn_cell.GRUCell(rnn_size)
state = tf.zeros([batch_size, rnn_cell.state_size])
X_split = tf.split(0, time_step_size, x_data)
outputs, state = tf.nn.rnn(rnn_cell, X_split, state)

# rnn_size = output의 vector size. num of classes(characters). 글자가 네종류면 4. 
# rnn의 time 갯수는 실행시킬 때 결정할 수 있다. 다섯 글자를 넣고 싶으면 다섯개를.
# training 결과로 state 값이 만들어지는데 나중에 학습할 때 이 state를 다시 사용할 수 있다.

In [5]:
print(rnn_cell.state_size)
print(rnn_cell.output_size)
print(rnn_cell.zero_state)

5
5
<bound method RNNCell.zero_state of <tensorflow.python.ops.rnn_cell.BasicRNNCell object at 0x11822ae48>>


In [6]:
logits = tf.reshape(tf.concat(1, outputs), [-1, rnn_size])
targets = tf.reshape(sample[1:], [-1])
weights = tf.ones([time_step_size * batch_size])

In [7]:
print(outputs)
print(logits.get_shape())
print(targets.get_shape())
print(weights.get_shape())

[<tf.Tensor 'RNN/BasicRNNCell/Tanh:0' shape=(1, 5) dtype=float32>, <tf.Tensor 'RNN/BasicRNNCell_1/Tanh:0' shape=(1, 5) dtype=float32>, <tf.Tensor 'RNN/BasicRNNCell_2/Tanh:0' shape=(1, 5) dtype=float32>, <tf.Tensor 'RNN/BasicRNNCell_3/Tanh:0' shape=(1, 5) dtype=float32>]
(4, 5)
(4,)
(4,)


In [8]:
loss = tf.nn.seq2seq.sequence_loss_by_example([logits], [targets], [weights])
cost = tf.reduce_sum(loss)/batch_size
train_op = tf.train.RMSPropOptimizer(0.01, 0.9).minimize(cost)

In [15]:
sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())

In [16]:
for i in range(100):
    sess.run(train_op)
    result = sess.run(tf.arg_max(logits, 1))
    print(sess.run(outputs))
    print(sess.run(state))
    print(result, [char_rdic[t] for t in result])
    print('----')

[array([[ 0.51766539, -0.1649487 ,  0.44794923, -0.38655668, -0.09477098]], dtype=float32), array([[-0.66039473, -0.29543209, -0.14321722,  0.69117248, -0.16413717]], dtype=float32), array([[ 0.58356249,  0.43662792, -0.31609544, -0.58478475,  0.3393392 ]], dtype=float32), array([[-0.47012514, -0.53548223, -0.42302558, -0.64917803, -0.49263632]], dtype=float32)]
[[-0.47012514 -0.53548223 -0.42302558 -0.64917803 -0.49263632]]
[0 3 0 2] ['h', 'o', 'h', 'l']
----
[array([[ 0.52039385, -0.15587451,  0.46250546, -0.38538092, -0.11594569]], dtype=float32), array([[-0.65375912, -0.29551092, -0.09860254,  0.69593024, -0.18914115]], dtype=float32), array([[ 0.58172256,  0.42539907, -0.26189128, -0.57572991,  0.30483976]], dtype=float32), array([[-0.46364138, -0.54012263, -0.37840796, -0.6251936 , -0.52182925]], dtype=float32)]
[[-0.46364138 -0.54012263 -0.37840796 -0.6251936  -0.52182925]]
[0 3 0 2] ['h', 'o', 'h', 'l']
----
[array([[ 0.52330446, -0.1468987 ,  0.47600213, -0.38377881, -0.136813

In [14]:
sess.close()